# Visualization Notebook

This notebook generates interactive visualizations from analysis results.

## Available Visualizations

- **Evaluation Dashboard** - 9-step analysis results
- **Semantic Network** - Theme relationships (force-directed graph)
- **Sentiment Chart** - Emotional flow visualization
- **Sankey Diagram** - Temporal flow

## Setup

In [ ]:
import sys
from pathlib import Path
import json

# Add framework to path
FRAMEWORK_ROOT = Path.cwd().parent.parent.parent
sys.path.insert(0, str(FRAMEWORK_ROOT))

# Import framework modules
from framework.core.ontology import AnalysisOutput
from framework.visualization import (
    EvaluationDashboardAdapter,
    ForceGraphAdapter,
    SentimentChartAdapter,
    SankeyAdapter,
)

print(f"Framework root: {FRAMEWORK_ROOT}")
print("Visualization adapters loaded!")

In [ ]:
# Configuration
PROJECT_DIR = Path.cwd().parent
PROJECT_NAME = PROJECT_DIR.name

PROCESSED_DIR = PROJECT_DIR / "data" / "processed"
VIZ_DIR = PROJECT_DIR / "visualizations"
VIZ_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project: {PROJECT_NAME}")
print(f"Analysis data: {PROCESSED_DIR}")
print(f"Output directory: {VIZ_DIR}")

## Helper Functions

In [ ]:
def load_analysis(name):
    """Load analysis data from processed directory."""
    file_path = PROCESSED_DIR / f"{name}_data.json"
    if not file_path.exists():
        print(f"❌ Not found: {file_path}")
        return None
    
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    return AnalysisOutput(
        module_name=data.get("module", name),
        data=data,
        metadata=data.get("metadata", {}),
    )

def list_available_analyses():
    """List available analysis files."""
    if not PROCESSED_DIR.exists():
        return []
    return [f.stem.replace("_data", "") for f in PROCESSED_DIR.glob("*_data.json")]

available = list_available_analyses()
print(f"\n📁 Available analyses: {available}")

## Generate Evaluation Dashboard

In [ ]:
# Load evaluation data
evaluation_data = load_analysis("evaluation")

if evaluation_data:
    print(f"✅ Loaded evaluation data")
    
    # Show summary
    summary = evaluation_data.data.get("summary", {})
    print(f"   Overall score: {summary.get('overall_score', 0):.1f}")
else:
    print("❌ Run evaluation analysis first: lingframe analyze -p <project> -m evaluation")

In [ ]:
# Generate evaluation dashboard
if evaluation_data:
    adapter = EvaluationDashboardAdapter()
    
    output_path = VIZ_DIR / "evaluation_dashboard.html"
    
    config = {
        "title": f"Evaluation Dashboard: {PROJECT_NAME}",
        "subtitle": "9-Step Rhetorical Analysis",
    }
    
    adapter.generate(evaluation_data, output_path, config)
    
    print(f"\n✅ Dashboard generated: {output_path}")
    print(f"\n   Open in browser: file://{output_path.absolute()}")

## Generate Semantic Network

In [ ]:
# Load semantic analysis
semantic_data = load_analysis("semantic")

if semantic_data:
    print(f"✅ Loaded semantic data")
    nodes = semantic_data.data.get("nodes", [])
    edges = semantic_data.data.get("edges", [])
    print(f"   Nodes: {len(nodes)}, Edges: {len(edges)}")

In [ ]:
# Generate force-directed graph
if semantic_data:
    adapter = ForceGraphAdapter()
    
    output_path = VIZ_DIR / "semantic_force_graph.html"
    
    config = {
        "title": f"Semantic Network: {PROJECT_NAME}",
        "subtitle": "Theme relationships and entity co-occurrence",
    }
    
    adapter.generate(semantic_data, output_path, config)
    
    print(f"\n✅ Network generated: {output_path}")

## Generate Sentiment Chart

In [ ]:
# Load sentiment analysis
sentiment_data = load_analysis("sentiment")

if sentiment_data:
    print(f"✅ Loaded sentiment data")
    sentences = sentiment_data.data.get("sentence_sentiments", [])
    print(f"   Sentences analyzed: {len(sentences)}")

In [ ]:
# Generate sentiment chart
if sentiment_data:
    adapter = SentimentChartAdapter()
    
    output_path = VIZ_DIR / "sentiment_sentiment_chart.html"
    
    config = {
        "title": f"Sentiment Analysis: {PROJECT_NAME}",
        "subtitle": "Emotional flow through the text",
    }
    
    adapter.generate(sentiment_data, output_path, config)
    
    print(f"\n✅ Sentiment chart generated: {output_path}")

## Generate All Visualizations

In [ ]:
# Batch generate all available visualizations

ADAPTERS = {
    "evaluation": (EvaluationDashboardAdapter, "evaluation_dashboard.html"),
    "semantic": (ForceGraphAdapter, "semantic_force_graph.html"),
    "sentiment": (SentimentChartAdapter, "sentiment_chart.html"),
    "temporal": (SankeyAdapter, "temporal_sankey.html"),
}

generated = []

for analysis_name, (adapter_cls, filename) in ADAPTERS.items():
    data = load_analysis(analysis_name)
    if data:
        try:
            adapter = adapter_cls()
            output_path = VIZ_DIR / filename
            adapter.generate(data, output_path)
            generated.append(filename)
            print(f"✅ {analysis_name} → {filename}")
        except Exception as e:
            print(f"❌ {analysis_name}: {e}")

print(f"\n📊 Generated {len(generated)} visualizations in {VIZ_DIR}")

## View Visualizations

To view the HTML visualizations, you have two options:

### Option 1: Local HTTP Server (Recommended)
```bash
cd <project_dir>/visualizations
python3 -m http.server 8000
```
Then open `http://localhost:8000/` in your browser.

### Option 2: Direct File Opening
Open the HTML files directly in your browser (some features may be limited due to CORS).

In [ ]:
# List generated files
print("\n📁 Generated Visualizations:")
print("=" * 50)

for f in VIZ_DIR.glob("*.html"):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:40} {size_kb:6.1f} KB")

print(f"\n📂 Directory: {VIZ_DIR.absolute()}")

## Inline Preview (Optional)

Display visualizations inline if running in Jupyter Lab or classic Notebook.

In [ ]:
# Inline HTML preview (requires IPython)
try:
    from IPython.display import IFrame, display, HTML
    
    # Preview evaluation dashboard (if generated)
    dashboard_path = VIZ_DIR / "evaluation_dashboard.html"
    if dashboard_path.exists():
        print("📊 Evaluation Dashboard Preview:")
        display(IFrame(src=str(dashboard_path), width=800, height=600))
    else:
        print("Dashboard not yet generated.")
except ImportError:
    print("IPython not available for inline preview.")
except Exception as e:
    print(f"Preview not available: {e}")
    print("Open the HTML files directly in your browser.")

---

## Custom Visualization Configuration

You can customize visualizations by passing configuration options.

In [ ]:
# Example: Custom evaluation dashboard
if evaluation_data:
    adapter = EvaluationDashboardAdapter()
    
    custom_config = {
        "title": "My Custom Analysis Report",
        "subtitle": "Detailed rhetorical breakdown",
        # Add more config options as needed
    }
    
    output_path = VIZ_DIR / "custom_dashboard.html"
    adapter.generate(evaluation_data, output_path, custom_config)
    
    print(f"✅ Custom dashboard: {output_path}")

---

## Next Steps

1. **Iterate on analysis**: Re-run with different configurations
2. **Share results**: The HTML files are self-contained and shareable
3. **Export data**: Use the JSON files for custom reports